In [1]:
import pandas as pd
import numpy as np
import csv
from pathlib import Path 
import re
import string

In [2]:
df= pd.read_csv(r'/Users/lolitalolita/Downloads/nutrients.csv')

In [3]:
df.Category.unique()

array(['Dairy products', 'Fats, Oils, Shortenings', 'Meat, Poultry',
       'Fish, Seafood', 'Vegetables A-E', 'Vegetables F-P',
       'Vegetables R-Z', 'Fruits A-F', 'Fruits G-P', 'Fruits R-Z',
       'Breads, cereals, fastfood,grains', 'Soups', 'Desserts, sweets',
       'Jams, Jellies', 'Seeds and Nuts', 'Drinks,Alcohol, Beverages'],
      dtype=object)

In [4]:
df=df.replace("t",0)
df=df.replace("t'",0)
df['Fiber']=df['Fiber'].replace("a","", regex=True)

In [5]:
df=df.replace(",","", regex=True)
df['Calories'][91]=(8+44)/2

In [6]:
df['Grams']=pd.to_numeric(df['Grams'])
df['Calories']=pd.to_numeric(df['Calories'])
df['Protein']=pd.to_numeric(df['Protein'])
df['Fat']=pd.to_numeric(df['Fat'])
df['Sat.Fat']=pd.to_numeric(df['Sat.Fat'])
df['Fiber']=pd.to_numeric(df['Fiber'])
df['Carbs']=pd.to_numeric(df['Carbs'])

In [7]:
df=df.dropna()

In [8]:
df

,Food,Measure,Grams,Calories,Protein,Fat,Sat.Fat,Fiber,Carbs,Category
0,Cows' milk,1 qt.,976,660.0,32,40,36.0,0.0,48.0,Dairy products
1,Milk skim,1 qt.,984,360.0,36,0,0.0,0.0,52.0,Dairy products
2,Buttermilk,1 cup,246,127.0,9,5,4.0,0.0,13.0,Dairy products
3,Evaporated undiluted,1 cup,252,345.0,16,20,18.0,0.0,24.0,Dairy products
4,Fortified milk,6 cups,1419,1373.0,89,42,23.0,1.4,119.0,Dairy products
...,...,...,...,...,...,...,...,...,...,...
330,Fruit-flavored soda,12 oz.,346,161.0,0,0,0.0,0.0,42.0,DrinksAlcohol Beverages
331,Ginger ale,12 oz.,346,105.0,0,0,0.0,0.0,28.0,DrinksAlcohol Beverages
332,Root beer,12 oz.,346,140.0,0,0,0.0,0.0,35.0,DrinksAlcohol Beverages
333,Coffee,1 cup,230,3.0,0,0,0.0,0.0,1.0,DrinksAlcohol Beverages


In [9]:
df.Measure.unique()

array(['1 qt.', '1 cup', '6 cups', '1 1/3 cups', '2/3 cup', '2 cups',
       '1/2 cup', '1-in. cube', '1 oz.', '2', '1T.', '1/4\xa0lb.', '1 T.',
       '2 slices', '3 oz.', '2 oz.', '1 pie', '3 1/2 oz.', '3 med.',
       '4 oz.', '5', '1 small', 'aver.', '6-8 med.', '1 steak', '1 large',
       '6 spears', '3/4 cup', '1 mad.', '1 ear', '8', '1/4 head', '4',
       '1', '6 small', '2 T.', '4 cups', '1 pod', '1 med.', '10 pieces',
       '10', '5 small', '1/3 cup', '1 med', '1/2 large', '1/2 med.', '3',
       '1/2', '6-oz. can', '8 oz. or', '1 large slice', 'I med.',
       '1 wedge', '1 slice', '1-lb. loaf', '1 serving', '2 med.',
       '1 section', '2 pieces', '2-oz. bar', '12 oz.'], dtype=object)

In [10]:
df.dtypes

Food         object
Measure      object
Grams         int64
Calories    float64
Protein       int64
Fat           int64
Sat.Fat     float64
Fiber       float64
Carbs       float64
Category     object
dtype: object

In [11]:
# Import PuLP modeler functions
from pulp import *

In [12]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("Minimal_sugar_intake", LpMinimize)

In [13]:
Name_dict = list(df['Food'])

Calories_dict = dict(zip(Name_dict, df['Calories']))
Protein_dict = dict(zip(Name_dict, df['Protein']))
Sat_Fat_dict = dict(zip(Name_dict, df['Sat.Fat']))
Fat_dict = dict(zip(Name_dict, df['Fat']))
Fiber_dict = dict(zip(Name_dict, df['Fiber']))
Carbs_dict = dict(zip(Name_dict, df['Carbs']))

In [14]:
# A dictionary called 'recipe_name_vars' is created to contain the referenced Variables
food_name_vars = LpVariable.dicts("R", Name_dict,lowBound=0,cat='Continuous')

In [15]:
# The constraints are added to 'prob'
# The objective function is added to 'prob' first

prob += (
    lpSum([Sat_Fat_dict[i] * food_name_vars[i] for i in Name_dict]),
    "Total sugar content per dish")

prob += (
    lpSum([Protein_dict[i] * food_name_vars[i] for i in Name_dict]) <= 70.0,
    "Protein_dict_Requirement_max",
)
prob += (
    lpSum([Protein_dict[i] * food_name_vars[i] for i in Name_dict]) >= 40.0,
    "Protein_dict_Requirement_min",
)

prob += (
    lpSum([Calories_dict[i] * food_name_vars[i] for i in Name_dict]) <= 2000.0,
    "Calories_dict_Requirement_max",
)
prob += (
    lpSum([Calories_dict[i] * food_name_vars[i] for i in Name_dict]) >= 1500.0,
    "Calories_dict_Requirement_min",
)

prob += (
    lpSum([Fat_dict[i] * food_name_vars[i] for i in Name_dict]) <= 97.0,
    "Fat_dict_Requirement_max",
)
prob += (
    lpSum([Fat_dict[i] * food_name_vars[i] for i in Name_dict]) >= 70.0,
    "Fat_dict_Requirement_min",
)

prob += (
    lpSum([Fiber_dict[i] * food_name_vars[i] for i in Name_dict]) <= 40.0,
    "Fiber_dict_Requirement_max",
)
prob += (
    lpSum([Fiber_dict[i] * food_name_vars[i] for i in Name_dict]) >= 25.0,
    "Fiber_dict_Requirement_min",
)

prob += (
    lpSum([Carbs_dict[i] * food_name_vars[i] for i in Name_dict]) <= 225.0,
    "Carbs_dict_Requirement_max",
)
prob += (
    lpSum([Carbs_dict[i] * food_name_vars[i] for i in Name_dict]) >= 180.0,
    "Carbs_dict_Requirement_min",
)



In [16]:
# The problem data is written to an .lp file
prob.writeLP("RecipeModel.lp")

[R_(1_2_cup_ice_cream),
 R_(cornstarch),
 R_3_teaspoons_sugar,
 R_9"_diam._pie,
 R_Almonds,
 R_Apple_betty,
 R_Apple_juice_canned,
 R_Apple_vinegar,
 R_Apples_raw,
 R_Apricots,
 R_Artichoke,
 R_Asparagus,
 R_Avocado,
 R_Bacon,
 R_Baked_with_cheese,
 R_Banana,
 R_Bean_soups,
 R_Bean_sprouts,
 R_Beans,
 R_Beef,
 R_Beef_soup,
 R_Beer,
 R_Beet_greens,
 R_Biscuits,
 R_Blackberries,
 R_Blueberries,
 R_Bouillon,
 R_Bran_flakes,
 R_Brazil_nuts,
 R_Bread_cracked_wheat,
 R_Bread_pudding,
 R_Broccoli,
 R_Brown_firm_packed_dark_sugar,
 R_Brussels_sprouts,
 R_Butter,
 R_Buttermilk,
 R_Cakes,
 R_Candied,
 R_Candy,
 R_Cane_Syrup,
 R_Cantaloupe,
 R_Carrots,
 R_Cashews,
 R_Cauliflower,
 R_Celery,
 R_Chard_steamed,
 R_Cheddar,
 R_Cheddar_grated_cup,
 R_Cheese,
 R_Cherries,
 R_Cherry_Pie,
 R_Chicken_livers_fried,
 R_Chocolate_creams,
 R_Chocolate_fudge,
 R_Chocolate_syrup,
 R_Clam_chowder,
 R_Clams,
 R_Cocoa,
 R_Cod,
 R_Coffee,
 R_Cola_drinks,
 R_Collards,
 R_Converted,
 R_Corn,
 R_Corn_bread_ground_meal

In [17]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/lolitalolita/opt/anaconda3/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/j9/7y5q0v6x4p3f2fdr5f5ffl0m0000gn/T/42c8d49b9f1347f083e27dc5fc15188c-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/j9/7y5q0v6x4p3f2fdr5f5ffl0m0000gn/T/42c8d49b9f1347f083e27dc5fc15188c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 2605 RHS
At line 2616 BOUNDS
At line 2617 ENDATA
Problem MODEL has 10 rows, 323 columns and 2450 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 9 (-1) rows, 316 (-7) columns and 2102 (-348) elements
Perturbing problem by 0.001% of 138.26748 - largest nonzero change 0.00096145185 ( 0.013220981%) - largest zero change 0.00094865208
0  Obj 0 Primal inf 27.277609 (4)
5  Obj 0.00068927932
Optimal - objective value 0
After Postsol

1

In [18]:
# Each of the variables is printed with it's resolved optimum value
    
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

R_Duck_domestic = 2.0171053
R_Gin = 2.4865789
R_Halibut = 1.3447368
R_Raw = 13.157895
R_Soda_2_1_2_square = 2.7631579


In [19]:
# The optimised objective function value is printed to the screen
print("Minimal sugar intake = ", value(prob.objective))

Minimal sugar intake =  0.0


Exploring the result.

Example: 
We want to check the measurement of each product to translate the result into a daily menu.
--> 1 serving of domestic duck is 3 1/2 oz. Following our recipe, the daily consumption of domestic duck would be  approx 7 oz. 
--> 1 serving of gin is 1 oz. Following our recipe, the daily consumption of gin would be  approx 2 1/2 oz. However,daily comsumption of alcohol is not good for anyone. This is one of the limitations of the algorithm, even if it is an optimal solution, it does not mean it is a good one. 



In [21]:
df[df['Food']=='Raw']

,Food,Measure,Grams,Calories,Protein,Fat,Sat.Fat,Fiber,Carbs,Category
216,Raw,1 cup,149,54.0,0,0,0.0,1.9,12.0,Fruits R-Z


In [22]:
df[df['Food']=='Gin']

,Food,Measure,Grams,Calories,Protein,Fat,Sat.Fat,Fiber,Carbs,Category
324,Gin,1 oz.,28,70.0,0,0,0.0,0.0,0.0,DrinksAlcohol Beverages


In [23]:
df[df['Food']=='Halibut']

,Food,Measure,Grams,Calories,Protein,Fat,Sat.Fat,Fiber,Carbs,Category
78,Halibut,3 1/2 oz.,100,182.0,26,8,0.0,0.0,0.0,Fish Seafood


In [28]:
df[df['Food']=='Duck domestic']

,Food,Measure,Grams,Calories,Protein,Fat,Sat.Fat,Fiber,Carbs,Category
59,Duck domestic,3 1/2 oz.,100,370.0,16,28,0.0,0.0,0.0,Meat Poultry


In [30]:
df[df['Food']=='Soda 2 1/2 square']

,Food,Measure,Grams,Calories,Protein,Fat,Sat.Fat,Fiber,Carbs,Category
231,Soda 2 1/2 square,2,11,45.0,1,1,0.0,0.0,8.0,Breads cereals fastfoodgrains
